In [1]:
import segmentation_models_pytorch as smp
from trainer import Trainer

In [2]:
MODEL = 'FPN'
ENCODER = 'efficientnet-b3'
OPTIMIZER = 'Ranger'         # options currently available : Over9000, Ralamb, RAdam, Ranger, LookaheadAdam
LOSS = 'TVERSKY'

In [3]:
def TRAIN(MODEL, ENCODER, OPTIMIZER, LOSS):
    if(MODEL == 'Unet'):
        model = smp.Unet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
    elif(MODEL == 'FPN'):
        model = smp.FPN(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
    elif(MODEL == 'Linknet'):
        model = smp.Linknet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
    
    model_trainer = Trainer(model = model, optim = OPTIMIZER, loss = LOSS, lr = 1e-3, bs = 8, name = ENCODER+'_'+MODEL+'_'+LOSS+'_'+OPTIMIZER)
    model.cutmix = False
    model_trainer.freeze()
    model_trainer.fit(10)
    model.cutmix = True
    model_trainer.unfreeze()
    model_trainer.fit(30)

In [ ]:
# Accordingly append to the list and train multiple models
# Naming has also been adjusted 
# ['Ranger','Over9000'] * ['Dice', 'BCE+DICE', 'BCE+DICE+JACCARD']
Model = ['Unet']
Encoder = ['efficientnet-b4']
Optimizer = ['Ranger', 'Over9000']
Loss = ['Dice', 'BCE+DICE', 'BCE+DICE+JACCARD']
for model in Model:
    for encoder in Encoder:
        for optimizer in Optimizer:
            for loss in Loss:
                TRAIN(model, encoder, optimizer, loss)
                    


Downloading: "https://publicmodels.blob.core.windows.net/container/aa/efficientnet-b4-6ed6700e.pth" to /home/aryan/.cache/torch/checkpoints/efficientnet-b4-6ed6700e.pth



Starting epoch: 0 | phase: train | ⏰: 13:25:52



Loss: 0.9469 | IoU: 0.0317 | dice: 0.0449 | dice_pos: 0.0706 | dice_neg: 0.0000 | f2_score: 0.0772
Starting epoch: 0 | phase: val | ⏰: 13:27:06



Loss: 0.9043 | IoU: 0.0393 | dice: 0.0661 | dice_pos: 0.1244 | dice_neg: 0.0000 | f2_score: 0.0950
* New optimal found, saving state *

Starting epoch: 1 | phase: train | ⏰: 13:27:16



Loss: 0.9300 | IoU: 0.0519 | dice: 0.0783 | dice_pos: 0.1240 | dice_neg: 0.0000 | f2_score: 0.1282
Starting epoch: 1 | phase: val | ⏰: 13:28:22



Loss: 0.9062 | IoU: 0.0499 | dice: 0.0974 | dice_pos: 0.1755 | dice_neg: 0.0000 | f2_score: 0.1410
* New optimal found, saving state *

Starting epoch: 2 | phase: train | ⏰: 13:28:30



Loss: 0.9078 | IoU: 0.0853 | dice: 0.1220 | dice_pos: 0.1891 | dice_neg: 0.0000 | f2_score: 0.1802
Starting epoch: 2 | phase: val | ⏰: 13:29:36



Loss: 0.9022 | IoU: 0.0713 | dice: 0.1193 | dice_pos: 0.2165 | dice_neg: 0.0000 | f2_score: 0.1715
* New optimal found, saving state *

Starting epoch: 3 | phase: train | ⏰: 13:29:44



Loss: 0.8782 | IoU: 0.1222 | dice: 0.1509 | dice_pos: 0.2296 | dice_neg: 0.0000 | f2_score: 0.2096
Starting epoch: 3 | phase: val | ⏰: 13:30:49



Loss: 0.8933 | IoU: 0.0761 | dice: 0.1204 | dice_pos: 0.2232 | dice_neg: 0.0090 | f2_score: 0.1693
* New optimal found, saving state *

Starting epoch: 4 | phase: train | ⏰: 13:30:58


In [ ]:
!pip3 install git+'https://github.com/pytorch/pytorch.git'

  Cloning https://github.com/pytorch/pytorch.git to /tmp/pip-req-build-xftgugrm
  Running command git clone -q https://github.com/pytorch/pytorch.git /tmp/pip-req-build-xftgugrm
  Running command git submodule update --init --recursive -q


In [2]:
# if(MODEL == 'Unet'):
#     model = smp.Unet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
# elif(MODEL == 'FPN'):
#     model = smp.FPN(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
# elif(MODEL == 'Linknet'):
#     model = smp.Linknet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)

In [ ]:
# from trainer import Trainer

In [ ]:
# model_trainer = Trainer(model = model, optim = OPTIMIZER, loss = LOSS, lr = 1e-3, bs = 8, name = ENCODER+'_'+MODEL+'_'+LOSS)

In [ ]:
model_trainer.seed_everything(43)

In [ ]:
# model_trainer.freeze()
# # model_trainer.fit(2)

In [ ]:
model_trainer.do_cutmix = False

In [ ]:
model_trainer.unfreeze()
model_trainer.fit(2)